In [183]:

import skimage.io
import skimage.color
import numpy as np

from pprint import pprint

In [184]:
import read_cifar10 as cf10

#@read_data.restartable
def cifar10_dataset_generator(dataset_name, batch_size, restrict_size=1000):
    assert dataset_name in ['train', 'test']
    assert batch_size > 0 or batch_size == -1  # -1 for entire dataset
    
    X_all_unrestricted, y_all = (cf10.load_training_data() if dataset_name == 'train'
                                 else cf10.load_test_data())
    
    actual_restrict_size = restrict_size if dataset_name == 'train' else int(1e10)
    X_all = X_all_unrestricted[:actual_restrict_size]
    data_len = X_all.shape[0]
    batch_size = batch_size if batch_size > 0 else data_len
    
    X_all_padded = np.concatenate([X_all, X_all[:batch_size]], axis=0)
    y_all_padded = np.concatenate([y_all, y_all[:batch_size]], axis=0)
    
    for slice_i in range(math.ceil(data_len / batch_size)):
        idx = slice_i * batch_size
        #X_batch = X_all_padded[idx:idx + batch_size]
        X_batch = X_all_padded[idx:idx + batch_size]*255  # bugfix: thanks Zezhou Sun!
        y_batch = np.ravel(y_all_padded[idx:idx + batch_size])
        yield X_batch.astype(np.uint8), y_batch.astype(np.uint8)

cifar10_dataset_generators = {
    'train': cifar10_dataset_generator('train', 1000),
    'test': cifar10_dataset_generator('test', -1)
}


In [185]:
#Load cifar-10 data

cf10_tr=cf10.load_training_data()
cf10_tr_img=cf10_tr[0]
cf10_tr_label = cf10_tr[1]
print(cf10_tr_img.shape)

cf10_test=cf10.load_test_data()
cf10_test_img=cf10_test[0]
cf10_test_label = cf10_test[1]
print(cf10_test_img.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [186]:
cf10_test_img_gray=(cf10_test_img[:,:,:,0]+cf10_test_img[:,:,:,1]+cf10_test_img[:,:,:,2])/3.
cf10_tr_img_gray=(cf10_tr_img[:,:,:,0]+cf10_tr_img[:,:,:,1]+cf10_tr_img[:,:,:,2])/3.
cf10_tr_vec=np.zeros((50000,1024))
cf10_test_vec=np.zeros((10000,1024))

In [366]:
for k in range(10):
    img=(255*cf10_test_img[k,:,:,:]).astype(np.uint8)
    skimage.io.imsave('../cifar10_jpg/'+str(k)+'.tiff',img)
    skimage.io.imsave('../cifar10_jpg/'+str(k)+'.jpg',img)


In [363]:
#Calculate the metrics for jpg


mse_jpg=np.zeros((10000,1))
psnr_jpg=np.zeros((10000,1))
#Save cifar test images
for k in range(10000):
    img_gray=(255*cf10_test_img_gray[k,:,:]).astype(np.uint8)
    skimage.io.imsave('../cifar10_jpg/'+str(k)+'.jpg',img_gray)
    
    img_vec = img_gray.reshape([1,-1])
    cf10_test_vec[k,:]=img_vec
    img_recons = skimage.io.imread('../cifar10_jpg/'+str(k)+'.jpg')
    #print(k)
    #print(img_recons)#-img_gray)
    #print(img_gray)
    #print(img_recons.astype(int)-img_gray.astype(int))
    #print(((img_recons.astype(float)-img_gray.astype(float))/255.)*255.)
    mse=(((img_recons.astype(float)-img_gray.astype(float))/255.)**2).mean()
    mse_jpg[k]=mse
    psnr_jpg[k]=10.*np.log10(1./mse)

print('test is done')
for k in range(50000):
    img_gray=(255*cf10_tr_img_gray[k,:,:]).astype(np.uint8)
    img_vec = img_gray.reshape([1,-1])
    cf10_tr_vec[k,:]=img_vec
    
print('train is done')



/home/mtezcan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:21: RuntimeWarning: divide by zero encountered in double_scalars


test is done
train is done


In [353]:
A= np.array([[1,2,3],[2,4,5]])
#print(np.array([[1,2,3]]).reshape([-1]).tolist() )

for k in range(9995,10000):
    if(cf10_test_vec[k,:].reshape([-1]).tolist() in cf10_tr_vec.tolist()):
        print(str(k)+':(')
    else:
        print(k)

9995
9996
9997
9998
9999


In [362]:
print(mse_jpg.mean())
print(psnr_jpg.mean())

0.000451892684004
33.7261858094


# Part-1 - MLP

In [8]:
x_tr = cf10_tr_vec.astype(np.float32)/255.
x_test = cf10_test_vec.astype(np.float32)/255.
x_test=x_test[:1000,:]
pprint(x_tr)
pprint(x_test)

array([[ 0.23921569,  0.17254902,  0.18431373, ...,  0.70588237,
         0.45882353,  0.37254903],
       [ 0.67450982,  0.51764709,  0.39607844, ...,  0.5411765 ,
         0.54509807,  0.54901963],
       [ 1.        ,  0.99215686,  0.99215686, ...,  0.31764707,
         0.32156864,  0.32549021],
       ..., 
       [ 0.58431375,  0.59215689,  0.60000002, ...,  0.20392157,
         0.16862746,  0.12156863],
       [ 0.83529413,  0.82352942,  0.81568629, ...,  0.71372551,
         0.72549021,  0.72549021],
       [ 0.90980393,  0.94117647,  0.93725491, ...,  0.6901961 ,
         0.63921571,  0.63529414]], dtype=float32)
array([[ 0.41568628,  0.41176471,  0.43137255, ...,  0.29411766,
         0.32156864,  0.25882354],
       [ 0.92156863,  0.90588236,  0.90980393, ...,  0.6901961 ,
         0.73725492,  0.7647059 ],
       [ 0.74509805,  0.73333335,  0.65098041, ...,  0.02352941,
         0.01568628,  0.02745098],
       ..., 
       [ 0.42745098,  0.51372552,  0.31764707, ...,  0.196

In [9]:
import tensorflow as tf

def mlp1(x, hidden_sizes, activation_fn=tf.nn.relu,dropout_rate=1.0,std_dev=1.0):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
    scope_args = {'initializer': tf.random_normal_initializer(stddev=std_dev)}
    for k in range(len(hidden_sizes)-1):
        layer_name="weights"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[k]])
            b = tf.get_variable('b', shape=[hidden_sizes[k]])
            x = activation_fn(tf.matmul(x, W) + b)
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes[-1]])
        return tf.matmul(x, W) + b

In [10]:
import tensorflow as tf

def mlp2(x, hidden_sizes_1,hidden_sizes_2, activation_fn=tf.nn.relu,dropout_rate=1.0,std_dev=1.0,cons_mult=1):
    scope_args = {'initializer': tf.random_normal_initializer(stddev=std_dev)}
    for k in range(len(hidden_sizes_1)-1):
        layer_name="weights_enc"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_1[k]])
            b = tf.get_variable('b', shape=[hidden_sizes_1[k]])
            x = activation_fn(tf.matmul(x, W) + cons_mult*b)
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer_enc', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_1[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes_1[-1]])
        x = activation_fn(tf.matmul(x, W) + cons_mult*b)
        
    x_quant = tf.round(x*255.)/255.
    
    for k in range(len(hidden_sizes_2)-1):
        layer_name="weights_dec"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_2[k]])
            b = tf.get_variable('b', shape=[hidden_sizes_2[k]])
            x = activation_fn(tf.matmul(x, W) + cons_mult*b)
            x_quant=(activation_fn(tf.matmul(x_quant, W) + cons_mult*b))
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer_dec', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_2[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes_2[-1]])
    
    return (tf.matmul(x, W) + cons_mult*b,tf.matmul(x_quant, W) + cons_mult*b)

In [11]:
import pprint

def test_classification(model_function, learning_rate=0.1):

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/gpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, 1024])
            #y_logits = model_function(x_)
            y_logits,x_recon = model_function(x_)

            
            # naive implementation of loss:
            # > losses = y_ * tf.log(tf.nn.softmax(y_logits))
            # > tf.reduce_mean(-tf.reduce_sum(losses, 1))
            # can be numerically unstable.
            #
            # so here we use tf.nn.softmax_cross_entropy_with_logits on the raw
            # outputs of 'y', and then average across the batch.
            
            loss = tf.reduce_mean(tf.subtract(x_,y_logits)**2)
            loss2=tf.reduce_mean(tf.subtract(x_,x_recon)**2)
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
            
            #y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            #correct_prediction = tf.equal(y_pred, tf.argmax(y_, 1))
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with g.as_default(), tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
        #sess.run(tf.initialize_all_variables())
        sess.run(tf.global_variables_initializer())
        
        # train
        ids=[i for i in range(100)]
        for iter_i in range(100001):
            batch_xs = x_tr[ids,:] 
            ids=[(ids[0]+100+i)%x_tr.shape[0] for i in range(100)]
            sess.run(train_step, feed_dict={x_: batch_xs})
            
            # test trained model
            if iter_i % 1000 == 0:
                tf_feed_dict = {x_: batch_xs}
                loss_val = sess.run(loss, feed_dict=tf_feed_dict)
                print('iteration %d\t train mse: %.3f\t'%(iter_i,loss_val))
                if iter_i%5000 == 0:
                    
                    loss_val_test = sess.run(loss, feed_dict={x_:x_test})
                    loss_val2_test = sess.run(loss2, feed_dict={x_:x_test})
                    print('iteration %d\t TEST MSE: %.3f\t %.3f\t'%(iter_i,loss_val_test,loss_val2_test))
                
                    
                    x_from_tr=sess.run(y_logits, feed_dict={x_:batch_xs[:5,:].reshape([-1,1024])})
                    x_from_test=sess.run(y_logits, feed_dict={x_:x_test[0:5,:].reshape([-1,1024])})
                    print('Train')
                    print(np.round(x_from_tr*255))
                    print(np.round(x_from_test*255))
                    print(np.round((x_from_tr-batch_xs[:5,:])*255))
                    print(np.round((x_from_test-x_test[:5,:])*255))
                    
                    
                
#test_classification(lambda x: mlp1(x, [850,700,500,700,850,1024],
#                                  activation_fn=tf.nn.relu,std_dev=1e-1), learning_rate=1e-3)

test_classification(lambda x: mlp2(x, [850,700,500],[700,850,1024],
                                  activation_fn=tf.nn.relu,std_dev=1,cons_mult=0.5), learning_rate=1e-3)

iteration 0	 train mse: 1273467466940416.000	
iteration 0	 TEST MSE: 1418206254202880.000	 1418206656856064.000	
Train
[[ -6.09862758e+09  -1.34699174e+09   5.41607834e+09 ...,   2.99168640e+09
   -4.81746534e+09   6.70333901e+09]
 [ -1.09867919e+10   4.78934688e+08   1.18972989e+10 ...,   9.97778330e+09
   -4.43372595e+09   6.46088384e+08]
 [ -7.13111245e+09   3.49363072e+09   1.06263368e+10 ...,   1.03928013e+10
   -6.47518259e+09   1.08516065e+10]
 [ -4.88430694e+09  -1.97268864e+09   3.95071309e+09 ...,   3.77766323e+09
   -2.98116173e+09   3.34111181e+09]
 [ -7.13384346e+09   7.20980992e+08   9.67383859e+09 ...,   1.00997540e+10
   -3.84320973e+09   2.96533171e+09]]
[[ -6.30383155e+09   1.10357210e+09   8.05959526e+09 ...,   7.24048691e+09
   -6.05417011e+09   6.51576883e+09]
 [ -9.74460723e+09   2.87364288e+08   9.29666253e+09 ...,   1.41880730e+10
   -1.33448819e+09   3.21045811e+09]
 [ -1.00041912e+10   2.25708570e+09   1.10657516e+10 ...,   1.04692593e+10
   -5.73511885e+09   

# Part-2 - CNN

In [231]:
x_tr = cf10_tr_img.astype(np.float32)#*255.
x_test = cf10_test_img.astype(np.float32)#*255.
x_test=x_test[:200,:,:,:]
print(x_tr.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(200, 32, 32, 3)


In [190]:

def unpool(x,unpool_size=[2,2], name='unpool'):
    with tf.name_scope(name) as scope:
        sh = x.get_shape().as_list()
        out=tf.image.resize_images(x_,[sh[1]*unpool_size[0],sh[2]*unpool_size[1]])
    return out

    

#x=np.random.rand(6,6)
#print(x[0:2:,0:2:])



print(x)
x_=tf.placeholder(tf.float32,[1,2,2,1])
#sh=x_.get_shape().as_list()
#sh=np.shape(x)
#dim = len(sh[1:-1])
#out = x.reshape([-1] + sh[-dim:])
#print(out)


y_=tf.placeholder(tf.float32,[1,4,4,1])
#y_=tf.image.resize_images(x_,[4,4])
#y_ = tf.nn.conv2d_transpose(x_, [2,2,1, 1], [1, 4,4, 1], [1, 2, 2, 1], padding='SAME', name=None)
y_=unpool(x_,unpool_size=[3,3])

x=np.array([[11.,2.],[2.,5.]]).reshape([1,2,2,1])
#print(x.shape())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    y=sess.run(y_,feed_dict={x_:x})
    print(y)

[[[[ 11.]
   [  2.]]

  [[  2.]
   [  5.]]]]
[[[[ 11.        ]
   [  8.        ]
   [  5.        ]
   [  2.        ]
   [  2.        ]
   [  2.        ]]

  [[  8.        ]
   [  6.33333349]
   [  4.66666651]
   [  3.        ]
   [  3.        ]
   [  3.        ]]

  [[  5.        ]
   [  4.66666651]
   [  4.33333349]
   [  4.        ]
   [  4.        ]
   [  4.        ]]

  [[  2.        ]
   [  3.        ]
   [  4.        ]
   [  5.        ]
   [  5.        ]
   [  5.        ]]

  [[  2.        ]
   [  3.        ]
   [  4.        ]
   [  5.        ]
   [  5.        ]
   [  5.        ]]

  [[  2.        ]
   [  3.        ]
   [  4.        ]
   [  5.        ]
   [  5.        ]
   [  5.        ]]]]


In [191]:
def cnn_map(x_,pool_size=[2,2],kernels=[5,7],filters=[16,128]):
    out_=x_
    for k in range(len(kernels)):
        conv = tf.layers.conv2d(inputs=out_,
                                filters=filters[k],
                                kernel_size=[kernels[k],kernels[k]],
                                padding="same",
                                activation=tf.nn.relu,
                                name='conv'+str(k))
        out_ = tf.layers.max_pooling2d(inputs=conv, 
                                       pool_size=pool_size, 
                                       strides=2,
                                       name = 'pool'+str(k))
    return out_

In [97]:
def decnn_map(x_,unpool_size=[2,2],kernels=[5,7],filters=[16,128]):
    #x_up=unpool(x_,unpool_size=unpool_size)
    for k in range(len(kernels)):
        x_up=unpool(x_,unpool_size=unpool_size,name='unpool'+str(k))
        x_ = tf.layers.conv2d(inputs=x_up,
                                filters=filters[k],
                                kernel_size=[kernels[k],kernels[k]],
                                padding="same",
                                activation=tf.nn.relu,
                                name='deconv'+str(k))
    return x_

In [232]:
def cnn_autoencoder(x_,kernels1=[5,7],kernels2=[7,5],filters1=[16,128],filters2=[128,3],pool_size=[1,2]):
    out_=x_
    for k in range(len(kernels1)):
        conv = tf.layers.conv2d(inputs=out_,
                                filters=filters1[k],
                                kernel_size=[kernels1[k],kernels1[k]],
                                padding="same",
                                activation=tf.nn.relu,
                                name='conv'+str(k))
        pool_now=pool_size[k]
        if(pool_now==1):
            out_=conv
        else:
            out_ = tf.layers.max_pooling2d(inputs=conv, 
                                           pool_size=[pool_now,pool_now], 
                                           strides=pool_now,
                                           name = 'pool'+str(k))
        
        out_quant=tf.round(out_*255.)/255.

    for k in range(len(kernels2)):
        with tf.variable_scope("deconv") as var_scope:
            pool_now=pool_size[-1-k]
            if(pool_now==1):
                x_up=out_
                out_ = tf.layers.conv2d(inputs=x_up,
                                        filters=filters2[k],
                                        kernel_size=[kernels2[k],kernels2[k]],
                                        padding="same",
                                        activation=tf.nn.relu,
                                        name='deconv'+str(k))
                var_scope.reuse_variables() 
                x_quant_up=out_quant
                out_quant = tf.layers.conv2d(inputs=x_quant_up,
                                            filters=filters2[k],
                                            kernel_size=[kernels2[k],kernels2[k]],
                                            padding="same",
                                            activation=tf.nn.relu,
                                            name='deconv'+str(k))
            else:
                sh = out_.get_shape().as_list()
                x_up=tf.image.resize_images(out_,[sh[1]*pool_now,sh[2]*pool_now])
                out_ = tf.layers.conv2d(inputs=x_up,
                                        filters=filters2[k],
                                        kernel_size=[kernels2[k],kernels2[k]],
                                        padding="same",
                                        activation=tf.nn.relu,
                                        name='deconv'+str(k))
                var_scope.reuse_variables() 
                x_quant_up=tf.image.resize_images(out_quant,[sh[1]*pool_now,sh[2]*pool_now])
                out_quant = tf.layers.conv2d(inputs=x_quant_up,
                                            filters=filters2[k],
                                            kernel_size=[kernels2[k],kernels2[k]],
                                            padding="same",
                                            activation=tf.nn.relu,
                                            name='deconv'+str(k))


    return out_,out_quant


In [324]:
#HVS matrix implementation
import scipy.fftpack as fft
f=np.linspace(1.0, 40.0, num=32)
h=2.6*(0.0192+0.114*f)*np.exp(-np.power((0.114*f),1.1)).reshape([1,-1])
H=np.matmul(np.ones((32,1)),h)
H2=(H*H.T).astype(np.float32)
print(H2)
h2 = fft.dct(fft.dct(H2,type=3,norm='ortho').T,type=3,norm='ortho')
h3=np.zeros((32,32,3,1)).astype(np.float32)
h3[:,:,0,0]=h2
h3[:,:,1,0]=h2
h3[:,:,2,0]=h2


print(h2)
print(h3)

[[ 0.09983069  0.18150099  0.23934336 ...,  0.02520034  0.02169322
   0.01864358]
 [ 0.18150099  0.32998478  0.43514732 ...,  0.04581644  0.03944018
   0.03389566]
 [ 0.23934336  0.43514732  0.57382399 ...,  0.06041763  0.05200934
   0.04469784]
 ..., 
 [ 0.02520034  0.04581644  0.06041763 ...,  0.00636134  0.00547604
   0.00470621]
 [ 0.02169322  0.03944018  0.05200934 ...,  0.00547604  0.00471394
   0.00405125]
 [ 0.01864358  0.03389566  0.04469784 ...,  0.00470621  0.00405125
   0.00348172]]
[[  1.00616789e+01   1.49537575e+00  -8.23408246e-01 ...,  -1.08088404e-02
   -6.29915297e-03  -7.97748566e-03]
 [  1.49537563e+00   2.22244024e-01  -1.22375660e-01 ...,  -1.60639174e-03
   -9.36212018e-04  -1.18559599e-03]
 [ -8.23408127e-01  -1.22375637e-01   6.73845112e-02 ...,   8.84542242e-04
    5.15500084e-04   6.52849674e-04]
 ..., 
 [ -1.08086262e-02  -1.60637801e-03   8.84536712e-04 ...,   1.16189622e-05
    6.77026401e-06   8.54581594e-06]
 [ -6.29931595e-03  -9.36207653e-04   5.15490

In [338]:
def apply_classification_loss(kernels1=[5,7],kernels2=[7,5],
                              filters1=[16,128],filters2=[128,3],
                              pool_size=[2,2],learning_rate=1.):
    with tf.Graph().as_default() as g:
        with tf.device("/gpu:0"):  # use gpu:0 if on GPU
            x_ = tf.placeholder(tf.float32, [None, 32, 32, 3])
            (x_out,x_out_quant)=cnn_autoencoder(x_,pool_size=pool_size,kernels1=kernels1,filters1=filters1,
                                kernels2=kernels2,filters2=filters2)

            #y_dict = dict(labels=y_, logits=y_logits)
            #losses = tf.nn.sparse_softmax_cross_entropy_with_logits(**y_dict)
            #cross_entropy_loss = tf.reduce_mean(losses)
            mse_loss1=tf.reduce_mean(tf.subtract(x_,x_out)**2)
            #a=tf.pad(tf.subtract(x_,x_out),[[0,0],[16,16],[16,16],[0,0]],'CONSTANT')

            #mse_loss1=tf.reduce_mean(tf.nn.conv2d(a,h3,strides=[1,1,1,1],padding="VALID")**2)
            mse_loss2=tf.reduce_mean(tf.subtract(x_,x_out_quant)**2)
            trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            train_op = trainer.minimize(mse_loss1)

            #y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            #correct_prediction = tf.equal(tf.cast(y_pred, tf.int32), y_)
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    model_dict = {'graph': g, 'inputs': x_,'outputs':x_out_quant, 'train_op': train_op, 'loss1': mse_loss1,'loss2': mse_loss2}
    
    return model_dict

In [367]:
def train_model(model_dict, dataset_generators, train_every=100, test_every=200,learning_rate=1.,fname='cifar10_recon'):
    with model_dict['graph'].as_default(), tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver=tf.train.Saver()
        ids=[i for i in range(100)]
        for iter_i in range(20000):
            batch_xs = x_tr[ids,:,:,:] 
            ids=[(ids[0]+100+i)%x_tr.shape[0] for i in range(100)]
            sess.run(model_dict['train_op'], feed_dict={model_dict['inputs']: batch_xs})
            
            # test trained model
            if iter_i % train_every == 0:
                tf_feed_dict = {x_: batch_xs}
                loss_val = sess.run(model_dict['loss1'], feed_dict={model_dict['inputs']: batch_xs})
                print('iteration %d\t train mse: %.3E\t'%(iter_i,loss_val))
                if iter_i % test_every == 0:
                    #tf_feed_dict = {x_: x_test}
                    loss_val1 = sess.run(model_dict['loss1'], feed_dict={model_dict['inputs']: x_test})
                    loss_val2 = sess.run(model_dict['loss2'], feed_dict={model_dict['inputs']: x_test})
                    print('iteration %d\t TEST MSE: %.3E\t TEST MSE(Quantized): %.3E\t'%(iter_i,loss_val1,loss_val2))
                    
                    
                    x_from_tr=sess.run(model_dict['outputs'], 
                                       feed_dict={model_dict['inputs']:batch_xs[:5,:,:,:].reshape([-1,32,32,3])})
                    x_from_test=sess.run(model_dict['outputs'], 
                                         feed_dict={model_dict['inputs']:x_test[:5,:,:,:].reshape([-1,32,32,3])})
                    
                    for i in range(5):
                        img_recon=(255*x_from_test[i,:,:,:]).reshape([32,32,3]).astype(np.uint8)
                        skimage.io.imsave('../'+fname+'/test'+str(i)+'_'+str(int(iter_i/test_every))+'.tiff',img_recon)
                        img_recon=(255*x_from_tr[i,:,:,:]).reshape([32,32,3]).astype(np.uint8)
                        skimage.io.imsave('../'+fname+'/tr'+str(i)+'_'+str(int(iter_i/test_every))+'.tiff',img_recon)
                    
                    print('Train')
                    print(np.round(x_from_tr*255).reshape([-1,3072]))
                    print(np.round((x_from_tr-batch_xs[:5,:])*255).reshape([-1,3072]))
                    print('Test')
                    print(np.round(x_from_test*255).reshape([-1,3072]))
                    print(np.round((x_from_test-x_test[:5,:])*255).reshape([-1,3072]))
        save_path = saver.save(sess, "/tmp/cnn_autoencoder.ckpt")
        print("Model saved in file: %s" % save_path)
        return saver
                


In [340]:
tf.reset_default_graph()
model_dict=apply_classification_loss(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,1,2],learning_rate=0.001)
saver = train_model(model_dict, [], train_every=100, test_every=500,fname='cifar10_recon0')

iteration 0	 train mse: 3.837E+01	
iteration 0	 TEST MSE: 4.245E+01	 TEST MSE(Quantized): 4.245E+01	
Train
[[ 301.  289.    0. ...,    0.  173.    0.]
 [ 462.  445.    0. ...,    0.  149.    0.]
 [ 757.  724.    0. ...,    0.  109.    0.]
 [ 224.  216.    0. ...,    0.  113.    0.]
 [ 389.  373.    0. ...,    0.  114.    0.]]
[[ 242.  227.  -63. ..., -123.   81.  -72.]
 [ 308.  268. -187. ..., -143.   16. -144.]
 [ 502.  469. -255. ...,  -80.   23.  -84.]
 [ 196.  191.  -10. ...,  -72.   48.  -46.]
 [ 219.  193. -198. ...,  -73.   37.  -80.]]
Test
[[ 413.  395.    0. ...,    0.  196.    0.]
 [ 702.  672.    0. ...,    0.  216.    0.]
 [ 582.  558.    0. ...,    0.   82.    0.]
 [ 572.  550.    0. ...,    0.  156.    0.]
 [ 273.  265.    0. ...,    0.  178.    0.]]
[[ 255.  283.  -49. ...,  -21.  129. -110.]
 [ 467.  437. -235. ..., -186.   16. -199.]
 [ 424.  368. -222. ...,   -7.   74.   -7.]
 [ 417.  394. -149. ...,  -73.   88.  -50.]
 [ 208.  197.  -50. ..., -128.   22. -117.]]
iter

KeyboardInterrupt: 

In [344]:
tf.reset_default_graph()
model_dict=apply_classification_loss(kernels1=[5,7],kernels2=[7,5],
                                     filters1=[16,128],filters2=[16,3],
                                     pool_size=[2,2],learning_rate=0.001)
saver = train_model(model_dict, [], train_every=100, test_every=500,fname='cifar10_recon1')

iteration 0	 train mse: 1.808E-01	
iteration 0	 TEST MSE: 2.088E-01	 TEST MSE(Quantized): 2.088E-01	
Train
[[  0.   7.  11. ...,  22.   0.   0.]
 [  0.  13.  16. ...,  19.   0.   0.]
 [  1.  16.  22. ...,  11.   0.   0.]
 [  0.   6.   8. ...,  14.   0.   0.]
 [  0.  11.  11. ...,  16.   0.   0.]]
[[ -59.  -55.  -52. ..., -101.  -92.  -72.]
 [-154. -164. -171. ..., -124. -133. -144.]
 [-254. -239. -233. ...,  -69.  -86.  -84.]
 [ -28.  -19.   -2. ...,  -58.  -65.  -46.]
 [-170. -169. -187. ...,  -57.  -77.  -80.]]
Test
[[  0.  10.  13. ...,  16.   0.   0.]
 [  1.  16.  22. ...,  22.   0.   0.]
 [  0.  14.  16. ...,  10.   0.   0.]
 [  2.  12.  18. ...,  19.   0.   0.]
 [  1.   7.   9. ...,  22.   0.   0.]]
[[-158. -102.  -36. ...,   -5.  -67. -110.]
 [-234. -219. -213. ..., -164. -200. -199.]
 [-158. -176. -206. ...,    3.   -8.   -7.]
 [-153. -144. -131. ...,  -54.  -68.  -50.]
 [ -64.  -61.  -41. ..., -106. -156. -117.]]
iteration 100	 train mse: 1.390E-02	
iteration 200	 train mse: 9

In [345]:
tf.reset_default_graph()
model_dict=apply_classification_loss(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,1,2],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=500,fname='cifar10_recon2')

iteration 0	 train mse: 1.825E-01	
iteration 0	 TEST MSE: 2.064E-01	 TEST MSE(Quantized): 2.064E-01	
Train
[[  0.  16.   0. ...,   0.   0.   2.]
 [  0.  30.   2. ...,   0.   0.   3.]
 [  0.  49.   0. ...,   0.   0.   1.]
 [  0.  12.   0. ...,   0.   0.   2.]
 [  0.  25.   1. ...,   0.   0.   3.]]
[[ -59.  -46.  -63. ..., -123.  -92.  -70.]
 [-154. -147. -185. ..., -143. -133. -141.]
 [-255. -206. -255. ...,  -80.  -86.  -83.]
 [ -28.  -13.  -10. ...,  -72.  -65.  -44.]
 [-170. -155. -197. ...,  -73.  -77.  -77.]]
Test
[[  0.  24.   0. ...,   0.   0.   3.]
 [  0.  45.   0. ...,   0.   0.   3.]
 [  0.  37.   1. ...,   0.   0.   2.]
 [  0.  36.   0. ...,   0.   0.   4.]
 [  0.  15.   0. ...,   0.   0.   3.]]
[[-158.  -88.  -49. ...,  -21.  -67. -107.]
 [-235. -190. -235. ..., -186. -200. -196.]
 [-158. -153. -221. ...,   -7.   -8.   -5.]
 [-155. -120. -149. ...,  -73.  -68.  -46.]
 [ -65.  -53.  -50. ..., -128. -156. -114.]]
iteration 100	 train mse: 1.488E-02	
iteration 200	 train mse: 6

In [368]:
tf.reset_default_graph()
model_dict=apply_classification_loss(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,2,1],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=500,fname='cifar10_recon3')

iteration 0	 train mse: 1.618E-01	
iteration 0	 TEST MSE: 1.833E-01	 TEST MSE(Quantized): 1.833E-01	
Train
[[  2.   7.   0. ...,   0.   4.   2.]
 [  7.  11.   0. ...,   0.   4.   2.]
 [ 11.  11.   0. ...,   0.   2.   0.]
 [  2.   4.   0. ...,   0.   3.   1.]
 [  6.   7.   0. ...,   0.   3.   0.]]
[[ -57.  -55.  -63. ..., -123.  -88.  -70.]
 [-147. -166. -187. ..., -143. -129. -142.]
 [-244. -244. -255. ...,  -80.  -84.  -84.]
 [ -26.  -21.  -10. ...,  -72.  -62.  -45.]
 [-164. -173. -198. ...,  -73.  -74.  -80.]]
Test
[[  5.   9.   0. ...,   0.   4.   0.]
 [ 10.  11.   0. ...,   0.   5.   2.]
 [  9.   9.   0. ...,   0.   3.   0.]
 [  7.  11.   0. ...,   1.   4.   2.]
 [  2.   6.   0. ...,   0.   4.   1.]]
[[-153. -103.  -49. ...,  -21.  -63. -110.]
 [-225. -224. -235. ..., -186. -195. -197.]
 [-149. -181. -222. ...,   -7.   -5.   -7.]
 [-148. -145. -149. ...,  -72.  -64.  -48.]
 [ -63.  -62.  -50. ..., -128. -152. -116.]]
iteration 100	 train mse: 1.076E-02	
iteration 200	 train mse: 7

In [349]:
tf.reset_default_graph()
model_dict=apply_classification_loss(kernels1=[5,7,9],kernels2=[9,7,5],
                                     filters1=[16,64,128],filters2=[64,16,3],
                                     pool_size=[1,1,2],learning_rate=0.0001)
saver = train_model(model_dict, [], train_every=100, test_every=500,fname='cifar10_recon4')

iteration 0	 train mse: 1.147E-01	
iteration 0	 TEST MSE: 1.316E-01	 TEST MSE(Quantized): 1.316E-01	
Train
[[  3.   0.   5. ...,   4.  10.  11.]
 [  5.   0.   8. ...,   1.   6.   9.]
 [  7.   0.  15. ...,   1.   6.   7.]
 [  2.   0.   3. ...,   1.   6.   7.]
 [  4.   0.   7. ...,   1.   5.   7.]]
[[ -56.  -62.  -58. ..., -119.  -82.  -61.]
 [-149. -177. -179. ..., -142. -127. -135.]
 [-248. -255. -240. ...,  -79.  -80.  -77.]
 [ -26.  -25.   -7. ...,  -71.  -59.  -39.]
 [-166. -180. -191. ...,  -72.  -72.  -73.]]
Test
[[  5.   0.   6. ...,   1.   7.  10.]
 [  7.   0.  13. ...,   3.  11.  13.]
 [  6.   0.  11. ...,   0.   3.   4.]
 [  6.   0.  10. ...,   1.   6.  10.]
 [  3.   0.   4. ...,   3.  11.  12.]]
[[-153. -112.  -43. ...,  -20.  -60. -100.]
 [-228. -235. -222. ..., -183. -189. -186.]
 [-152. -190. -211. ...,   -7.   -5.   -3.]
 [-149. -156. -139. ...,  -72.  -62.  -40.]
 [ -62.  -68.  -46. ..., -125. -145. -105.]]
iteration 100	 train mse: 7.394E-03	
iteration 200	 train mse: 4

In [315]:
#op = tf.shape(tf.nn.conv2d(tf.random_normal([1,10,10,3]), 
#             tf.random_normal([10,10,3,1]), 
#              strides=[1, 1,1, 1], padding='VALID'))
op = tf.pad(tf.random_normal([1,3,3,3]),[[0,0],[0,3],[0,3],[0,0]],'CONSTANT')

with tf.Session() as sess:
    result = sess.run(op)
    print(result.shape)

(1, 6, 6, 3)


In [178]:

with tf.Session() as sess:
    # Restore variables from disk.
    #sess.run(tf.global_variables_initializer())
    saver.restore(sess, "/tmp/cnn_autoencoder.ckpt")
    print("Model restored.")
    x_from_test=sess.run(model_dict['outputs'], 
                                feed_dict={model_dict['inputs']:x_test[:5,:,:,:].reshape([-1,32,32,3])})
    print(x_from_test)

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().